In [63]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [64]:
df = pd.read_csv('traindata2.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,2,48,193,0,0,12,12,2,...,0,1.360000,4.480000,25.000000,4.480000,1.360000,0.407407,0.776316,1.0,1.000000
1,0,2,0,82,193,0,1,17,27,5,...,1,1.533333,1.533333,15.000000,1.533333,1.533333,0.610169,0.452381,0.5,0.500000
2,0,2,0,66,193,0,2,17,19,5,...,0,3.959391,3.045685,3.283333,3.045685,3.959391,0.545455,0.717391,1.0,1.000000
3,0,2,2,82,193,0,2,12,16,5,...,1,1.733333,2.533333,15.000000,2.533333,1.733333,0.703125,0.611940,1.0,0.888889
4,0,2,1,48,193,0,2,14,26,0,...,0,5.454545,5.454545,0.183333,5.454545,5.454545,0.000000,0.500000,1.0,1.000000


In [65]:
dfMock = pd.read_csv('mockTestData.csv')
dfMock.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,redCorner_fightTime,blueCorner_fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,1,1,12.0,12.0,2.0,4.0,0.0,0.0,35.0,...,3.02,4.55,9.833333,13.733333,3.01,3.76,0.55,0.58,0.33,0.72
1,0,1,0,17.0,19.0,5.0,4.0,0.0,0.0,33.0,...,4.38,3.37,10.600000,11.416667,3.33,2.91,0.54,0.62,0.74,0.73
2,0,1,1,12.0,16.0,5.0,5.0,0.0,1.0,37.0,...,4.34,5.20,15.583333,11.100000,5.19,2.23,0.55,0.65,0.77,0.92
3,0,1,0,13.0,15.0,6.0,8.0,0.0,0.0,33.0,...,5.68,6.59,10.766667,12.950000,4.90,6.43,0.63,0.51,0.43,0.69
4,0,1,1,17.0,13.0,0.0,6.0,0.0,0.0,29.0,...,3.07,4.34,15.000000,13.266667,5.40,3.27,0.61,0.54,0.00,0.62


In [66]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6155
6155


In [67]:
print(len(dfMock))
dfMock = dfMock.dropna(inplace=False)
print(len(dfMock))

5092
5092


In [68]:
#dropping referee
df.drop('referee', axis=1, inplace=True)

#drop billings
df.drop('billing', axis=1, inplace=True)

#one hot encode redCorner_nation
df_encoded = pd.get_dummies(dfMock, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
df.drop('redCorner_nation', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_nation
df_encoded = pd.get_dummies(dfMock, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
df.drop('blueCorner_nation', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

#drop venue
df.drop('venue', axis=1, inplace=True)

#drop title_fight
df.drop('title_fight', axis=1, inplace=True)


df.head()


,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_103.0,blueCorner_nation_107.0,blueCorner_nation_110.0,blueCorner_nation_112.0,blueCorner_nation_113.0,blueCorner_nation_114.0,blueCorner_nation_115.0,blueCorner_nation_117.0,blueCorner_nation_118.0,blueCorner_nation_119.0
0,0,2,2,12,12,2,4,0,0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,0,17,27,5,12,1,0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,0,17,19,5,4,0,0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,2,12,16,5,5,0,1,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,1,14,26,0,8,0,0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
df['blueCorner'] = 1
df = df[df['winner'] != 1]
df['winner'] = df['winner'].replace(2, 1)


df.head()

ValueError: cannot reindex on an axis with duplicate labels

In [366]:
#x,y split
target_column = 'winner'
y = df[target_column]
X = df.drop(target_column, axis=1)

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()



,redCorner,blueCorner,title_fight,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,venue_196,venue_197,venue_198,venue_201,venue_202,venue_205,venue_206,venue_207,venue_208,venue_209
5980,0,1,0,8,6,2,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0
5799,0,1,0,22,6,8,1,0,0,29,...,0,0,0,0,0,0,0,0,0,0
1521,0,1,0,15,8,2,1,0,1,24,...,0,0,0,0,0,0,0,0,0,0
2343,0,1,0,23,15,8,5,0,0,42,...,0,0,0,0,0,0,0,0,0,0
1669,0,1,0,13,12,5,5,0,0,38,...,0,0,0,0,0,0,0,0,0,0


In [367]:
#Define the deep learning model
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

#earlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)


#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(X_train, y_train, epochs=1000, batch_size=124, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
37/39 [===========================>..] - ETA: 0s - loss: 0.5633 - accuracy: 0.7225
Epoch 1: val_accuracy improved from -inf to 0.76285, saving model to best_model_weights.h5
39/39 [==============================] - 2s 12ms/step - loss: 0.5587 - accuracy: 0.7248 - val_loss: 0.4520 - val_accuracy: 0.7629
Epoch 2/1000
39/39 [==============================] - ETA: 0s - loss: 0.3848 - accuracy: 0.8434
Epoch 2: val_accuracy improved from 0.76285 to 0.84992, saving model to best_model_weights.h5
39/39 [==============================] - 0s 10ms/step - loss: 0.3848 - accuracy: 0.8434 - val_loss: 0.3412 - val_accuracy: 0.8499
Epoch 3/1000
38/39 [============================>.] - ETA: 0s - loss: 0.3269 - accuracy: 0.8627
Epoch 3: val_accuracy did not improve from 0.84992
39/39 [==============================] - 0s 6ms/step - loss: 0.3296 - accuracy: 0.8621 - val_loss: 0.3333 - val_accuracy: 0.8483
Epoch 4/1000
27/39 [===================>..........] - ETA: 0s - loss: 0.3099 - accuracy

In [368]:
#accuracy
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_test,y_hat)

#0.8864013266998342

38/38 [==============================] - 0s 3ms/step


0.8814262023217247

In [369]:
#variable weights
first_layer_weights = model.layers[0].get_weights()[0]
input_variables = X_train.columns
variable_weights = dict(zip(input_variables, np.abs(first_layer_weights.flatten())))
ranked_variables = sorted(variable_weights.items(), key=lambda x: x[1], reverse=True)
print("Ranked Variables:")
for variable, weight in ranked_variables:
    print(f"{variable}: {weight}")

Ranked Variables:
blueCorner_nation_50: 0.21174356341362
blueCorner_nation_6: 0.16472549736499786
venue_120: 0.14383138716220856
venue_57: 0.13589636981487274
venue_162: 0.12582555413246155
redCorner_nation_58: 0.12574870884418488
blueCorner_nation_13: 0.12402216345071793
redCorner_nation_95: 0.12378573417663574
blueCorner_nation_26: 0.12047754228115082
blueCorner_nation_2: 0.1198096051812172
blueCorner_nation_93: 0.11854516714811325
blueCorner_nation_55: 0.11648006737232208
venue_142: 0.11283749341964722
venue_30: 0.11148937046527863
venue_11: 0.11033229529857635
blueCorner_nation_44: 0.10973824560642242
venue_133: 0.10907673090696335
blueCorner_subs_attempted: 0.10896721482276917
redCorner_nation_22: 0.1074802577495575
redCorner: 0.10747494548559189
venue_127: 0.10713514685630798
venue_143: 0.10581628978252411
blueCorner_nation_46: 0.1058102548122406
redCorner_draws: 0.10473790764808655
venue_139: 0.10439343750476837
redCorner_nation_88: 0.10430562496185303
redCorner_nation_51: 0.104